In [8]:
!pip uninstall opencv-python -y
# downgrade OpenCV a bit since some none-free features are not avilable
!pip install opencv-contrib-python==3.4.2.17 --force-reinstall


Uninstalling opencv-python-4.1.2.30:
  Successfully uninstalled opencv-python-4.1.2.30
     |████████████████████████████████| 30.6MB 144kB/s 
     |████████████████████████████████| 14.5MB 332kB/s 
ERROR: imgaug 0.2.9 requires opencv-python, which is not installed.
ERROR: dopamine-rl 1.0.5 requires opencv-python>=3.4.1.15, which is not installed.
ERROR: albumentations 0.1.12 requires opencv-python, which is not installed.
ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
  Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
path='/content/drive/My Drive/Septimo semestre/Procesamiento de imagen digital/detección de imágenes/Nintendo'
os.chdir(path)
!pwd
!ls

/content/drive/My Drive/Septimo semestre/Procesamiento de imagen digital/detección de imágenes/Nintendo
corazon   mario      pickmin	   prueba2.jpg	prueba6.jpg
link	  mario.jpg  pickmin.jpg   prueba3.jpg	prueba7.jpg
link.jpg  mario.wav  pickmin.wav   prueba4.jpg	prueba8.jpg
link.wav  photo.jpg  prueba10.jpg  prueba5.jpg	prueba9.jpg


In [ ]:

import cv2
import matplotlib.pyplot as plt
import numpy as np
from itertools import product
from IPython.display import Audio
from scipy.io import wavfile

%matplotlib inline

# Load the image
p = take_photo()#tomamos una foto de la camara
image1 = cv2.imread('photo.jpg')#leemos la foto 
training_image = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)#cambiamos su modelo de color de BGR a RGB
training_gray = cv2.cvtColor(training_image, cv2.COLOR_RGB2GRAY)#cambiamos su modelo de color de RGB a escala de grises
bordes = cv2.Canny(training_gray, 90, 200)# encontramos los bordes de la imagen
training_gray2 = cv2.GaussianBlur(training_gray, (7, 7), 3)#pasamos un filtro blur 


_, contours, _ = cv2.findContours(bordes, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)#encontramos los contornos de la imagen

areas = [cv2.contourArea(c) for c in contours]# encontramos el contorno mas grande
areaMax= np.argmax(areas)

mask = np.zeros_like(training_gray) # creamos una mascara del tamaño de la imagen
cv2.drawContours(mask, contours, areaMax, 255, -1)
out = np.zeros_like(training_gray) # enmascaramos la seccion de interes, que en este caso son las cartas
out[mask == 255] = training_gray[mask == 255]

(x, y) = np.where(mask == 255)
(topx, topy) = (np.min(x), np.min(y))# recortamos la imagen para que solo quede la parte de la camara
(bottomx, bottomy) = (np.max(x), np.max(y))
out = out[topx:bottomx+1, topy:bottomy+1]

des,car = sacarCaracterisricas(out)#encontramos las caracterisicas y el descriptor de la imagen recortada

print(car)
mat=[];
num = 0
fig = 0
lmayor = 0

ranks = ['mario','link','pickmin']
#for s, r in product(suits, ranks): 
for r in ranks:#recorremos el arreglo con las cartas
    stringImg =r+'.jpg'
    print(stringImg)  
    l = compararImg(des,r)#buscamos matches entre la imagen de prueba y la foto que tomamos
  
    if r=='pickmin':
       l=l*1.01 #hacemos un pequeño ajuste a la cantidad de matches con la imagen de prueba de los pickmin
    print(l)
    if l>=lmayor:
      lmayor=l
      num=r#encontramos la carta con mayor cantidad de matches 
      #fig=s
print("la carta es "+num)#la imprimimos 
imagen = cv2.imread(num+'.jpg')

imagenFinal = cv2.cvtColor(imagen, cv2.COLOR_BGR2RGB)

fx, plots = plt.subplots(1, 2, figsize=(20,10))

plots[0].set_title("Foto de entrada")
plots[0].imshow(out)

plots[1].set_title("Carta identificada")
plots[1].imshow(imagenFinal)

data = wavfile.read(num+".wav")

framerate = data[0]
sounddata = data[1]
time = np.arange(0,len(sounddata))/framerate

Audio(sounddata,rate=framerate,autoplay=True)


<IPython.core.display.Javascript object>

AttributeError: ignored

In [3]:
def compararImg(dO,s):
    #sift = cv2.xfeatures2d.SIFT_create() # intanciamos algoritmo sift
    #surf = cv2.xfeatures2d.SURF_create() # intanciamos algoritmo surf
    #orb = cv2.ORB_create()  # intanciamos algoritmo orb
    Odescriptor = dO
    Pdescriptor = load(s)#cargamos el descriptor de la imagen de entrenamiento correspondiente
    #Pkeypoints, Pdescriptor = surf.detectAndCompute(imgP, None)#encontramos los descriptores y caracteristicas de la imagen de entrenamiento
    #save(s,Pdescriptor)#guardamos en un archivo el descriptor 
    #print(r+s)
    bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck = True)#instanciamos el objeto que va a calcular los martches
    
    matches = bf.match(Odescriptor, Pdescriptor)#encontramos los matches entre la foto recortada y la imagen de entrenamiento
    matches = sorted(matches, key = lambda x : x.distance)#ajustamos los matches para dejar los mas cercanos

    return len(matches)
    
def sacarCaracterisricas(img):
    #sift = cv2.xfeatures2d.SIFT_create()
    surf = cv2.xfeatures2d.SURF_create()
    #orb = cv2.ORB_create()
    Okeypoints, Odescriptor = surf.detectAndCompute(img, None)
    return Odescriptor,len(Okeypoints)

In [2]:
import pickle
def save(filename,object):
    file=open(filename,'wb')#abrimos el archivo filename si no existe se crea
    pickle.dump(object,file)#sobreescribimos el archivo
    file.close()#cerramos y guardamos el archivo
    print("lo guarde")

def load(filename):
    file=open(filename,'rb')
    object=pickle.load(file)#traemos los datos dentro del archivo y lo guardamos en la variable object
    file.close()
    return object

In [41]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

#Codigo de:
#https://colab.research.google.com/notebooks/snippets/advanced_outputs.ipynb

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename